In [ ]:
#Importing all the relevant libraries 
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.datasets import cifar10
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
#Loading the data from the cifar10 dataset
(X_train,y_train), (X_test,y_test) = cifar10.load_data()

In [ ]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

In [ ]:
#Examining the first image in the training set which is a frog (given label 6)
first_image = X_train[0]
plt.imshow(first_image)
print(first_image)
print(y_train[0])

In [ ]:
#First we need to normalize the dataset since the range of values are between 0 and 255

X_train_norm = X_train/255
X_test_norm = X_test/255

#Create a validation set from the training set 

validation_proportion = 0.2
training_data_size = int(len(X_train_norm)*(1-validation_proportion))

X_final_train, y_final_train = X_train_norm[:training_data_size], y_train[:training_data_size]
X_val, y_val = X_train_norm[training_data_size:], y_train[training_data_size:]

In [ ]:
#Creating a callback to end training early if the accuracy reaches 90% or more

class Callback(tf.keras.callbacks.Callback):
    
    def on_epoch_end(self, epoch, logs=None):
        if logs.get('accuracy') >= 0.9:         
            print("\nStopping training early. Accuracy has reached 90%")
            self.model.stop_training = True

In [ ]:
#Build our model to train

callback = Callback()

model = tf.keras.Sequential()

#INPUT 
model.add(layers.Conv2D(32,3,input_shape=(32,32,3), kernel_regularizer=tf.keras.regularizers.l1(l1=0.01)))
model.add(layers.MaxPool2D((2,2)))
model.add(layers.Flatten())

#HIDDEN
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dropout(0.2))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dropout(0.2))

#OUTPUT
model.add(layers.Dense(10,activation='softmax'))

print(model.summary())

model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

history = model.fit(X_final_train, y_final_train, epochs=50, callbacks=[callback], validation_data=(X_val, y_val))

In [ ]:
history_dict = history.history
loss = history_dict['loss']
accuracy = history_dict['accuracy']
val_loss = history_dict['val_loss']
val_accuracy = history_dict['val_accuracy']

epochs = np.array(range(len(loss))) + 1

fig, (ax1,ax2) = plt.subplots(1,2)

fig.set_figheight(5)
fig.set_figwidth(15)

ax1.plot(epochs, loss, label="Training loss")
ax1.plot(epochs,val_loss,label="Validation loss")

ax1.set(xlabel="Epoch",ylabel="Loss")

ax1.legend()

ax2.plot(epochs, accuracy, label="Training accuracy",color='green')
ax2.plot(epochs,val_accuracy,label="Validation accuracy",color='magenta')

ax2.legend()

ax2.set(xlabel="Epoch",ylabel="Acccuracy")


In [ ]:
#EVALUATE OUR MODEL ON THE TEST SET

loss, accuracy = model.evaluate(X_test_norm, y_test)

print("The loss on the test set is {loss}".format(loss=loss))
print("The accuracy on the test set is {accuracy}".format(accuracy=accuracy))

In [ ]:
#Evaluating the model on the first test image
plt.imshow(X_test[0])
print("The predicted class is {value}".format(value=model.predict(np.array([X_test_norm[0]])).argmax()))
print("The actual class is {value}".format(value=y_test[0]))

In [ ]:
plt.imshow(X_test[4])
print("The predicted class is {value}".format(value=model.predict(np.array([X_test_norm[4]])).argmax()))
print("The actual class is {value}".format(value=y_test[4]))

In [ ]:
plt.imshow(X_test[10])
print("The predicted class is {value}".format(value=model.predict(np.array([X_test_norm[10]])).argmax()))
print("The actual class is {value}".format(value=y_test[10]))